In [36]:
# import shit

import torch
import torchtext
from torchtext.vocab import Vectors, GloVe
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from matplotlib import pyplot as plt

import numpy as np

In [37]:
# Our input $x$
TEXT = torchtext.data.Field()

# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

print('len(train)', len(train))

TEXT.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))

train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=10, device=-1, bptt_len=32, repeat=True)

len(train) 1
len(TEXT.vocab) 10001


In [38]:
#function to calculate perplexity
def perplexity(pred, true):
    nll = nn.NLLLoss()
    loss = nll(pred,true)
    perplexity = np.exp(loss.data/pred.size()[0])
    

In [89]:
#LSTM that predicts based on last hidden state
class LCA_LSTM(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, batch_size, layers = 2):
        
        super(LCA_LSTM,self).__init__()
        
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.layers = layers
        
        #define layers of our model
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        
        #self.embedding.weight.data = TEXT.vocab.vectors
        #self.embedding.weight.requires_grad = False
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,layers,dropout=0.5)
        self.hidden = self.init_hidden()
        self.final = nn.Linear(hidden_dim, vocab_size)
        self.d1 = nn.Dropout(p=0.5)
        self.d2 = nn.Dropout(p=0.5)
        
        self.fresh = True
        
    def init_hidden(self):
        return (Variable(torch.zeros(self.layers,self.batch_size,self.hidden_dim).cuda()),Variable(torch.zeros(self.layers,self.batch_size,self.hidden_dim).cuda()))
    
    def forward(self, sentence):
        
        #get embedded vectors
        vectors = self.embedding(sentence)
        
        #vectors = self.d1(vectors)
        
        #clean hidden layer
        if self.fresh:
            self.hidden = self.init_hidden()
        
        #pass through the entire sentence
        seq, self.hidden = self.lstm(vectors, tuple(state.detach() for state in self.hidden))
        
        #project from hidden state to word space
        scores = [self.final(self.d1(h_t)) for h_t in seq]
        
        scores = torch.cat(scores)
        
        return scores

In [ ]:
#function to train lstm model
def train_lstm(best_model):
    
    #get certain constants
    vocab_size = len(TEXT.vocab)
    train_len = len(train_iter)
    val_len = len(val_iter)
    epochs = 20
    pad_idx = TEXT.vocab.stoi["<pad>"]
    batch_size = 10
    
    #create model
    model = LCA_LSTM(500, 200, len(TEXT.vocab), batch_size, 3).cuda()
    
    loss_function = nn.CrossEntropyLoss(size_average=True)
    loss_function2 = nn.NLLLoss(size_average=False)
    #optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.05)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = 0.001)
    
    for epoch in range(epochs):
        
        model.fresh = True
        
        num_train_batches = 0
        total_train_loss = 0.0
            
        for batch in train_iter:
                      
            if num_train_batches is not 0:
                model.fresh = False
                
            num_train_batches += 1  
            
            with torch.cuda.device(0):
                model.zero_grad()
            
                log_probs = model(batch.text.cuda())
            
                loss = loss_function(log_probs, batch.target.view(-1).cuda())
            
                #retain graph if its not the last batch
                loss.backward(retain_graph=(num_train_batches is not train_len))
                optimizer.step()
            
            total_train_loss += loss.data
            
            if num_train_batches >= train_len:
                break
        
        #set fresh to wipe hidden state for validation run
        model.fresh = True
        
        #accumulate loss then divide by number of batches
        num_val_batches = 0.0
        num_val_words = 0.0
        total_val_loss = 0.0
        total_val_nll = 0.0
        
        #calculate loss and perplexity on language model
        for batch in val_iter:
            
            non_pad = batch.target.ne(pad_idx).cuda().view(-1)
            
            log_probs = model(batch.text.cuda())
            
            loss = loss_function(log_probs, batch.target.cuda().view(-1))
            
            total_val_loss += loss.data[0]
            
            non_pad_probs = log_probs[non_pad.nonzero(),:]
            non_pad_true = batch.target.cuda().view(-1)[non_pad]
                
            loss2 = loss_function(non_pad_probs.squeeze(), non_pad_true)

            total_val_nll += loss2.data[0]
            
            num_val_words += batch.target.ne(pad_idx).sum().data[0]
            num_val_batches += 1
                
            if num_val_batches >= val_len:
                break
            
        #calculate perplexity
        perp = np.exp((total_val_nll)/num_val_batches)
        
        #report statistics
        print("Epoch %d: Avg Val Loss (X-E): %f, Val Perplexity: %f, Avg Train Loss: %f" % (epoch,total_val_loss/num_val_batches,perp,total_train_loss/num_train_batches))
        
        if perp < best_model['perp']:
            best_model['model'] = model
            best_model['perp'] = perp

    return model

my_dict = {'model':None, 'perp':np.inf}
my_model = train_lstm(my_dict)

In [74]:
_2lstm=my_dict['model']

LCA_LSTM(
  (embedding): Embedding(10001, 300)
  (lstm): LSTM(300, 50, num_layers=2)
  (final): Linear(in_features=50, out_features=10001)
  (d1): Dropout(p=0.5)
  (d2): Dropout(p=0.5)
)

In [29]:
a = next(iter(train_iter))

In [50]:
torch.ByteTensor([1,0]).nonzero()


 0
[torch.LongTensor of size 1x1]

In [20]:
np.exp(3)

20.085536923187668